In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Reshape, Input, Flatten
from sklearn.model_selection import train_test_split

In [2]:
import time, datetime
from fyers_apiv3 import fyersModel
client_id = "ZHQ4IJL7TI-100"
with open("access_token", "r") as f:
    access_token = f.read()

fyers = fyersModel.FyersModel(client_id=client_id, is_async=False, token=access_token, log_path="")

In [3]:
import matplotlib.pyplot as plt
import numpy as np

In [20]:
from indicators.syncind import SyncInd
from indicators.classic import SMA, Alligator, SmoothMA, RSI, MACD, ATR, WMA, EMA, RMA, VolumeROC, KAMA
from indicators.candle import OHLC
from indicators import epoch_to_time

In [5]:
date_yesterday = (datetime.datetime.now() - datetime.timedelta(days=1)).strftime("%Y-%m-%d")
date_100_p = (datetime.datetime.now() - datetime.timedelta(days=100)).strftime("%Y-%m-%d")
date_yesterday, date_100_p

('2024-12-28', '2024-09-20')

In [6]:
response = fyers.history(data={
                                "symbol": "NSE:NIFTY50-INDEX",
                                "resolution": "1",
                                "date_format": "1",
                                "range_from": date_100_p,
                                "range_to": date_yesterday,
                                "cont_flag": "1"
                                })

len(response['candles'])

24810

In [7]:
date_temp = []
final_candles_list = []
for i in response['candles']:
    if i[0] not in date_temp:
        final_candles_list.append(i)
        date_temp.append(i[0])
del date_temp
len(final_candles_list)

24810

In [8]:
candles = np.array(final_candles_list)
candles.shape

(24810, 6)

In [9]:
sync = SyncInd(
    RSI(14)
)

for c in candles:
    sync.append(c)
sync.data().shape

(24810, 7)

In [10]:
sync.data().shape

(24810, 7)

In [12]:
sync.data()[-1, -1]

60.751598813871105

In [26]:
profit_list = []
back_test_days = -5000

for indx, i in enumerate(sync.data()[back_test_days:, -1]):
    j = sync.data()[back_test_days + (indx + 1), -1]
    if i < 30 and j > 30:
        trade_open = sync.data()[back_test_days+indx+1, 3]
        k = 1
        while i < 70:
            i = sync.data()[back_test_days+indx+k, -1]
            k = k + 1
        trade_close = sync.data()[back_test_days+indx+k-1, 3]
        profit = trade_close - trade_open
        print("trade_id:", len(profit_list), "\ntrade_open:", epoch_to_time(sync.data()[back_test_days+indx+1, 0]), "\ttrade_close:", epoch_to_time(sync.data()[back_test_days+indx+k-1, 0]))
        profit_list.append(profit)
        print("profit:", round(profit, 2), end="\n\n")
print("average profit:", sum(profit_list)/len(profit_list))

trade_id: 0 
trade_open: 2024-12-09 14:21:12 	trade_close: 2024-12-09 14:46:48
profit: 27.6

trade_id: 1 
trade_open: 2024-12-09 14:25:28 	trade_close: 2024-12-09 14:46:48
profit: 30.7

trade_id: 2 
trade_open: 2024-12-10 10:39:20 	trade_close: 2024-12-10 14:29:44
profit: -63.05

trade_id: 3 
trade_open: 2024-12-10 11:49:44 	trade_close: 2024-12-10 14:29:44
profit: -77.45

trade_id: 4 
trade_open: 2024-12-10 11:58:16 	trade_close: 2024-12-10 14:29:44
profit: -73.45

trade_id: 5 
trade_open: 2024-12-10 12:15:20 	trade_close: 2024-12-10 14:29:44
profit: -23.15

trade_id: 6 
trade_open: 2024-12-10 12:38:48 	trade_close: 2024-12-10 14:29:44
profit: -0.25

trade_id: 7 
trade_open: 2024-12-10 12:43:04 	trade_close: 2024-12-10 14:29:44
profit: 10.55

trade_id: 8 
trade_open: 2024-12-11 12:15:20 	trade_close: 2024-12-13 11:11:20
profit: -389.55

trade_id: 9 
trade_open: 2024-12-11 14:46:48 	trade_close: 2024-12-13 11:11:20
profit: -369.65

trade_id: 10 
trade_open: 2024-12-11 14:53:12 	trade_c